<a href="https://colab.research.google.com/github/irohappa/iro/blob/master/Gemini_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このプログラムを実行する前に、GeminiのAPIキーを発行してシークレットキーに保管してください
またNDCのドキュメントをドライブに保存してください

NDCのtxtデータはEmbeddingのレポジトリに落ちてます
(余裕があったらRAGのレポジトリにもおいておきますね)

In [41]:
#必要なライブラリのインストール
!pip install langchain
!pip install -q -U google-generativeai

In [42]:
pip install -U langchain-community

In [43]:
# for text model
model = genai.GenerativeModel('gemini-pro')

In [44]:
#Google Driveへのアクセス許可
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
#必要なライブラリインポート
# Embedding用
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
# テキストファイルを読み込む
from langchain.document_loaders import TextLoader

In [46]:
# 環境変数の準備
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [47]:
txt_path = "/content/drive/MyDrive/doc_class.txt"

In [48]:
#テキストデータの読み込み
loader = TextLoader(txt_path)
documents = loader.load()

In [49]:
#テキストデータの分割
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [50]:
import pandas as pd

# Extract content and metadata from Document objects
data = []
for doc in docs:
    data.append({
        "doc": doc.page_content,
        "source": doc.metadata.get('source'), # Assuming 'source' is in metadata
        "type": None # You might need to determine the type based on your data
    })

df = pd.DataFrame(data, columns=["doc", "source", "type"])
display(df)

,doc,source,type
0,国立国会図書館\n「日本十進分類法（NDC）新訂 10 版」\n分類基準\n\n令和 4 年...,/content/drive/MyDrive/doc_class.txt,None
1,目次\n本基準の凡例 ......................................,/content/drive/MyDrive/doc_class.txt,None
2,本基準の凡例\n\n\n各項目には、分類基準に関連する分類記号、分類項目名、注記などを、N...,/content/drive/MyDrive/doc_class.txt,None
3,\n\n固有補助表を使用しない場合は、個別に明記する。\n\n1\n\n 本表・補助表編\...,/content/drive/MyDrive/doc_class.txt,None
4,情報産業．情報サービス\n\n個々の企業史・誌も，ここに収める；特定主題の情報サービス業は，...,/content/drive/MyDrive/doc_class.txt,None
...,...,...,...
215,団体，企業体．会社誌，研究調査機関，教育・養成機関\n\nこれらの記号は，原則として－02 ...,/content/drive/MyDrive/doc_class.txt,None
216,分を使用し，地理区分を行わない；ただし，項目名が団体に該当する場合，団体を表す形式\n区分は...,/content/drive/MyDrive/doc_class.txt,None
217,115\n\n －088 資料集\n特定主題についての史料集に，使用する\n－02 と－08...,/content/drive/MyDrive/doc_class.txt,None
218,相関索引・使用法編\n使用法\nI 2.1 4）複数主題\nもし 4 以上の主題を取り扱い，...,/content/drive/MyDrive/doc_class.txt,None


In [51]:
#分割したドキュメントをEmbedding
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

docs=list(docs)
result = genai.embed_content(
    model="models/embedding-001",
    content=df["doc"],
    task_type="retrieval_document",
    title="Embedding of list of strings")
df["Embedding"]=result["embedding"]

In [52]:
#得られたベクトルを標準化
import numpy as np
def normalize_vector(v):
    norm = np.linalg.norm(v)
    if norm == 0:
       return v
    return v / norm

df["Embedding"]=df["Embedding"].map(normalize_vector)
passage_embeddings=df["Embedding"]

In [53]:
#queryをEmbedding
result = genai.embed_content(
    model="models/embedding-001",
    content="""
    手宮線は、北海道では最初に開通した路線です。
    """,
    task_type="retrieval_document",
    title="Embedding of list of strings")
query_embedding = normalize_vector(result["embedding"])

In [54]:
#得られたベクトルの内積を計算
scores = []
for content in passage_embeddings:
  scores.append(np.dot(content,query_embedding))

In [55]:
#内積を降順に並べ替えた時のインデックスを取得
def get_sorted_indices(lst):
    return [i for i, v in sorted(enumerate(lst), key=lambda x: x[1],reverse = True)]
scores_indices = get_sorted_indices(scores)

In [56]:
#コサイン類似度の高い３つの文章を出力
print(df["doc"][scores_indices[0:3]])

191    体育医学．スポーツ医学\n\n＊スポーツ障害＜一般＞は，ここに収める\nスポーツ障害全般に関...
215    団体，企業体．会社誌，研究調査機関，教育・養成機関\n\nこれらの記号は，原則として－02 ...
138    電波伝播．無線回路・測定\n\n無線工学の観点から扱われた電離層は，ここに収める\n地球物理...
Name: doc, dtype: object


次に実際にLLMを通して効果を確認してみます

今回使うのはLlama2を日本語に関して教科学習させた
"elyza/ELYZA-japanese-Llama-2-7b-instruct"

In [57]:
# パッケージのインストール
!pip install transformers accelerate bitsandbytes

In [58]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import tempfile

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct"
)
# Create a temporary directory for offloading
offload_dir = tempfile.mkdtemp()
model = AutoModelForCausalLM.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder=offload_dir
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [59]:
#プロンプトを作成する
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """
あなたは誠実で優秀な日本人のアシスタントです。
与えられた文章に対して適切な分類項目を答えてください
この際分類項目に関する情報が与えられます
"""

def q(text):
	prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
	    bos_token=tokenizer.bos_token,
	    b_inst=B_INST,
	    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
	    prompt=text,
	    e_inst=E_INST,
	)
	with torch.no_grad():
	    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
	    output_ids = model.generate(
	        token_ids.to(model.device),
	        max_new_tokens=1024,
	        pad_token_id=tokenizer.pad_token_id,
	        eos_token_id=tokenizer.eos_token_id,
	    )
	output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
	print(output)

In [ ]:
#RAGする前の出力
text = """手宮線は、北海道では最初に開通した路線ですよ。
    """
q(text)

In [ ]:
#RAGしてきた文章を元のプロンプトに加える
query = text +str(df["doc"][scores_indices[0]])
print(query)

In [ ]:
q(query)